In [1]:
# set run environment (local/colab), if colab move proper dir
import os
from pathlib import Path

if os.getenv("COLAB_RELEASE_TAG"):
    colab = True
    
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/Othercomputers/My computer/EQILLM/
    
    !pip install -r requirements.txt -q --exists-action i
    !pip install transformers[torch] -q --exists-action i
    !pip install accelerate -U -q --exists-action i
else:
    colab = False

import csv
import datetime
import evaluate
import glob
import itertools
import numpy as np
import openai
import pandas as pd
import torch, gc
import wandb

from datasets import Dataset, load_dataset
from dotenv import load_dotenv, dotenv_values
from huggingface_hub import login
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from tqdm.auto import tqdm
from tqdm.notebook import tqdm_notebook
from transformers import AutoModelForSequenceClassification, Trainer, BitsAndBytesConfig, AutoTokenizer, DataCollatorWithPadding

from eqillm import finetune, yeelight_eow_notification, param_combinations, load_PolarIs, split_ds, encode_labels, init_model

dotenv_config = dotenv_values('.env')
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if ('YEELIGHT_NOTIFY' in dotenv_config) and (colab) else False

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
login(token=dotenv_config['HF_TOKEN'])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Jakub\.cache\huggingface\token
Login successful


In [2]:
# models = ['albert', 'bart', 'bert', 'big_bird', 'bigbird_pegasus', 'biogpt', 'bloom', 'camembert', 'canine', 'code_llama', 'convbert', 'ctrl', 'data2vec-text', 'deberta', 'deberta-v2', 'distilbert', 'electra', 'ernie', 'ernie_m', 'esm', 'falcon', 'flaubert', 'fnet', 'funnel', 'gemma', 'gpt-sw3', 'gpt2', 'gpt_bigcode', 'gpt_neo', 'gpt_neox', 'gptj', 'ibert', 'layoutlm', 'layoutlmv2', 'layoutlmv3', 'led', 'lilt', 'llama', 'longformer', 'luke', 'markuplm', 'mbart', 'mega', 'megatron-bert', 'mistral', 'mixtral', 'mobilebert', 'mpnet', 'mpt', 'mra', 'mt5', 'mvp', 'nezha', 'nystromformer', 'open-llama', 'openai-gpt', 'opt', 'perceiver', 'persimmon', 'phi', 'plbart', 'qdqbert', 'qwen2', 'reformer', 'rembert', 'roberta', 'roberta-prelayernorm', 'roc_bert', 'roformer', 'squeezebert', 'stablelm', 'starcoder2', 't5', 'tapas', 'transfo-xl', 'umt5', 'xlm', 'xlm-roberta', 'xlm-roberta-xl', 'xlnet', 'xmod', 'yoso',]
# =============================================
# works = [']
# ----------
#too weak pc (cannot be loaded or runs at speeds <1 it/s) = 'albert-xlarge-v1', 't5-11B', 'LongformerForSequenceClassification', 'xlm-roberta-base', 'allenai/longformer-base-4096', 'facebook/bart-large',  flaubert/flaubert_large_cased (stopped at 0.5 epoch),
# ----------
# need changes to run (check error logs) =  ProsusAI/finbert (despite adding padding proposed in error)
# =============================================
# 'xlnet-base-cased' - needs data collator

# , 'camembert-base', 'ctrl', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased'

# deprecated: transfo-xl-wt103


# ['michellejieli/emotion_text_classifier', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased']



# to_solve


In [5]:
MAX_NEW_TOKENS = 256

params_tested = {'model_name': [
                              'microsoft/phi-2',
                            'google/gemma-2b',
                            'mistralai/Mistral-7B-v0.1',
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'michellejieli/emotion_text_classifier',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'roberta-base',
                           # 'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           # 'ProsusAI/finbert',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'camembert-base'
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'ctrl',
                           # 'camembert-base',
                           # 'papluca/xlm-roberta-base-language-detection',
                           # 'mistralai/Mistral-7B-v0.1',
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'meta-llama/Meta-Llama-3-8B',
                           # 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'arpanghoshal/EmoRoBERTa',
                           # 'ProsusAI/finbert',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'michellejieli/emotion_text_classifier',
                           # 'xlnet-base-cased',
                           # 'roberta-base',
                           # 'distilroberta-base',
                           # 'flaubert/flaubert_base_cased',
                           #  'celine98/canine-s-finetuned-sst2',
                           #  'lytang/MiniCheck-Flan-T5-Large',
                           #  'michelecafagna26/t5-base-finetuned-sst2-sentiment',
                           #  'facebook/tart-full-flan-t5-xl',
                           #  'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           #  'nlptown/bert-base-multilingual-uncased-sentiment',
                           #  'cardiffnlp/twitter-xlm-roberta-base-sentiment',
                           ], # Pre-trained model names from the Hugging Face hub used for fine-tuning
                 'num_train_epochs': 10, # Number of times the model sees the entire training dataset.
                 'save_strategy': 'epoch', # Controls when to save model checkpoints ('epoch' or 'no').
                 'per_device_train_batch_size': 16, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                 'per_device_eval_batch_size': 16, # Number of samples processed in each evaluation step.
                 'split': (0.8, 0.1, 0.1), # Divides the dataset into training, testing, (and optionally) validation sets. Use 'balanced' for equal class representation in the validation set. Examples: (90,10) -> split into train and test proportionally; (80, 10, 10) splits into train,test, validate proportionally.
                 'binary': [True], # Indicates whether the task is binary (two classes) or multi-class classification.,
                 'balanced': [False], # his way labels used for training are split evenly, fitting size to the lowest label count. n (equal to 80% of least represented label) will be taken from each label, rest will be used for test.
                 'learning_rate': [
                                    # 1e-3, 
                                    1e-4, 
                                    # 1e-5,
                                    ],
                 }

# Controls whether to save logs during a process. When set to False, logging is disabled.
save_logs = True

train_params_looped = param_combinations(params_tested)
print(len(train_params_looped))
print(*train_params_looped, sep='\n')

3
{'model_name': 'microsoft/phi-2', 'num_train_epochs': 10, 'save_strategy': 'epoch', 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'split': (0.8, 0.1, 0.1), 'binary': True, 'balanced': False, 'learning_rate': 0.0001}
{'model_name': 'google/gemma-2b', 'num_train_epochs': 10, 'save_strategy': 'epoch', 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'split': (0.8, 0.1, 0.1), 'binary': True, 'balanced': False, 'learning_rate': 0.0001}
{'model_name': 'mistralai/Mistral-7B-v0.1', 'num_train_epochs': 10, 'save_strategy': 'epoch', 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'split': (0.8, 0.1, 0.1), 'binary': True, 'balanced': False, 'learning_rate': 0.0001}


In [6]:
from transformers import TrainingArguments

data_path = 'data/PolarIs-Pathos.xlsx'

#run looped
for train_params in tqdm(train_params_looped):
    os.environ["WANDB_PROJECT"]="pathos-recognition-binary"

    df = load_PolarIs(data_path, train_params['binary'])
    df, target_map = encode_labels(df)
    ds = Dataset.from_pandas(df)
    ds = split_ds(ds, train_size=0.8, val_size=0.1)

    finetune(ds, train_params, target_map, colab)

if yeelight_notify:
    yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])

  0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/12470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Map:   0%|          | 0/1560 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Map:   0%|          | 0/1560 [00:00<?, ? examples/s]

torch.float16


C:\Users\Jakub\DataspellProjects\EQILLM\venv\Lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 134218752.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
# import datasets
# from transformers import pipeline
# from transformers.pipelines.pt_utils import KeyDataset
# from tqdm.auto import tqdm
#
# pipe = pipeline("text-classification", model="output/models/google/gemma-2b_2024-04-27_00-13/checkpoints/checkpoint-140290", device=0)
# valds = df_to_ds(polarIs_df)[0]['validate']
# predicted = [i['label'] for i in pipe(valds['text'])]